In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
d={
    'id':pd.Series(data=[1,2,3,4,5,6]),
    'name':pd.Series(data=['Ira','Jayesh','Amita','Prashant','Neeraj',np.nan]),
    'age':pd.Series(data=[5,25,31,41,23,100]),
    'city':pd.Series(data=['Pune','Hyderabad','Pune','Pune','Mumbai','Mumbai'])
}

data = pd.DataFrame(d,columns=['id','name','age','city'])
data

,id,name,age,city
0,1,Ira,5,Pune
1,2,Jayesh,25,Hyderabad
2,3,Amita,31,Pune
3,4,Prashant,41,Pune
4,5,Neeraj,23,Mumbai
5,6,NaN,100,Mumbai


In [3]:
#    = != > >= < <=
#    
#    like
#    notlike
#    
#    startsWith
#    endsWith
#    
#    between
#    notbetween
#    
#    in
#    notin
#    
#    isnull
#    notisnull

In [4]:
def fun(attr,oper,val):
    attr = "data['"+attr+"']"
    
    if(oper == 'like'):
        oper = ".str.contains('"+val+"', regex=True)"
        string = '('+attr+oper+')'
    elif(oper == 'notlike'):
        oper = ".str.contains('"+val+"', regex=True)"
        string = '~('+attr+oper+')'
    elif(oper == 'startsWith'):
        oper = ".str.contains('^"+val+"', regex=True)"
        string = '('+attr+oper+')'
    elif(oper == 'endsWith'):
        oper = ".str.contains('"+val+"$', regex=True)"
        string = '('+attr+oper+')'
        
    elif(oper == 'in'):
        in_list = val.split(',')
        int_list = [i.isdigit() for i in in_list]
        if(all(int_list)):
            in_string = val
        else:
            in_string = ', '.join("'" + item + "'" for item in in_list)
        oper = ".isin(["+in_string+"])"
        string = '('+attr+oper+')'
    elif(oper == 'notin'):
        in_list = val.split(',')
        int_list = [i.isdigit() for i in in_list]
        if(all(int_list)):
            in_string = val
        else:
            in_string = ', '.join("'" + item + "'" for item in in_list)
        oper = ".isin(["+in_string+"])"
        string = '~('+attr+oper+')'
        
    elif(oper == 'between'):
        low,up = val.split(',')
        string = '((' + attr + '>=' + low + ') & (' + attr + '<=' + up + '))'
    elif(oper == 'notbetween'):
        low,up = val.split(',')
        string = '((' + attr + '<' + low + ') | (' + attr + '>' + up + '))'
    
    elif(oper == 'isnull'):
        oper = ".isnull()"
        string = '('+attr+oper+')'
    elif(oper == 'notisnull'):
        oper = ".isnull()"
        string = '~('+attr+oper+')'
    
    else:
         string = '(' + attr + " " + oper + " " + val +')'
    return string

In [8]:
js = [
    {'attribute':'id', 'operator':'>=', 'value':'2'},
    {'attribute':'city', 'operator':'in', 'value':'Pune,Mumbai'},
    {'attribute':'id', 'operator':'notin', 'value':'1,2'},
    {'attribute':'age', 'operator':'notbetween', 'value':'30,40'},
    {'attribute':'city', 'operator':'notlike', 'value':'ai'},
    {'attribute':'name', 'operator':'notisnull'}
]
json_df = df = pd.DataFrame(data=js,columns=['attribute','operator','value'])
json_df['string'] = json_df.apply(lambda x: fun(x['attribute'],x['operator'],x['value']), axis=1)

json_df

,attribute,operator,value,string
0,id,>=,2,(data['id'] >= 2)
1,city,in,"Pune,Mumbai","(data['city'].isin(['Pune', 'Mumbai']))"
2,id,notin,"1,2","~(data['id'].isin([1,2]))"
3,age,notbetween,"30,40",((data['age']<30) | (data['age']>40))
4,city,notlike,ai,"~(data['city'].str.contains('ai', regex=True))"
5,name,notisnull,NaN,~(data['name'].isnull())


In [9]:
condition = ' & '.join(json_df['string'])
condition

"(data['id'] >= 2) & (data['city'].isin(['Pune', 'Mumbai'])) & ~(data['id'].isin([1,2])) & ((data['age']<30) | (data['age']>40)) & ~(data['city'].str.contains('ai', regex=True)) & ~(data['name'].isnull())"

In [10]:
data[eval(condition)]

,id,name,age,city
3,4,Prashant,41,Pune
